In [1]:
# Load dependencies
import findspark
findspark.init()
import pyspark
import random
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext()
sc.version

'2.2.0'

In [2]:
import pandas as pd
import scipy.stats

In [23]:
scipy.stats.mode([1,2,2,3])[0][0]

2

In [4]:
# Import raw data
sqlContext = SQLContext(sc)
transactions = sqlContext.read.csv('../../data/raw/transactions.csv',header=True)
user_logs = sqlContext.read.csv('../../data/raw/user_logs.csv',header=True)
train = sqlContext.read.csv('../../data/raw/train.csv',header=True)
#test = sqlContext.read.csv('../../data/raw/sample_submission_zero.csv',header=True)
members = sqlContext.read.csv('../../data/raw/members.csv',header=True)

## 1. check the dimension of each table

In [7]:
print((transactions.count(), len(transactions.columns)))

(21547746, 9)


In [ ]:
print((user_logs.count(), len(user_logs.columns))) # it's just too big to even getting two numbers here ...

In [3]:
print((train.count(), len(train.columns)))

(992931, 2)


In [4]:
print((test.count(), len(test.columns))) # why train set is even as big as test set? 

(970960, 2)


In [5]:
print((members.count(), len(members.columns)))

(5116194, 7)


Yes, it is. Beacuse training set records all users whose service gonna expire in Feb. Test set records all users whose service gonna expire in Mar. It's against common intuition in machine learning data splitting. 

# 2. Join training set with the smallest table 

In [5]:
df_train = train.join(members, on=['msno'], how='left_outer')

In [17]:
df_train.head(3)

[Row(msno='++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=', is_churn='0', city='1', bd='0', gender=None, registered_via='7', registration_init_time='20140714', expiration_date='20170930'),
 Row(msno='+/HS8LzrRGXolKbxRzDLqrmwuXqPOYixBIPXkyNcKNI=', is_churn='0', city=None, bd=None, gender=None, registered_via=None, registration_init_time=None, expiration_date=None),
 Row(msno='+/namlXq+u3izRjHCFJV4MgqcXcLidZYszVsROOq/y4=', is_churn='0', city='15', bd='30', gender='male', registered_via='9', registration_init_time='20060603', expiration_date='20170930')]

## 3. Tables Aggregation : Transaction + user_log  

### (1) Transaction Table

In [25]:
transactions.printSchema()

root
 |-- msno: string (nullable = true)
 |-- payment_method_id: string (nullable = true)
 |-- payment_plan_days: string (nullable = true)
 |-- plan_list_price: string (nullable = true)
 |-- actual_amount_paid: string (nullable = true)
 |-- is_auto_renew: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- membership_expire_date: string (nullable = true)
 |-- is_cancel: string (nullable = true)



In [14]:
transactions.head(1)

[Row(msno='YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=', payment_method_id='41', payment_plan_days='30', plan_list_price='129', actual_amount_paid='129', is_auto_renew='1', transaction_date='20150930', membership_expire_date='20151101', is_cancel='0')]

There are 8 features in this table and I plan to do some simple feature aggregation here to reduce redundancy, as well as the data size.I will group the table by each user("msno")and create 4 aggregation features :

- common_payment_method: Compute the mode of **"payment_method_id"** to represnt the most common payment method of a user
- common_plan_days:Compute the mode of **"payment_plan_days"** to represnt the most common plan period of a user 
- median_plan_price: Compute the median of **"plan_list_price"** to represnt the average plan price a user often chose 
- cancel_rate: Compute the mean of **"is_canel"** to represnt the actual frequency of membership cancellation of a user

Here I ignored the information of 4 features.

- "actual_amount_paid": It seems to contain the same information as feature "plan_list_price"
- "is_auto_renew" : It is an OR condition of "is_cancel".
- "transaction_date" and "membership_expire_date": These specific date information could be replaced by the feature "payment_plan_days".

In [ ]:
.select([c for c in df.columns if c not in {'GpuName','GPU1_TwoPartHwID'}])

--- Logging error ---
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:63192)
Traceback (most recent call last):
  File "C:\Users\dell-pc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-1b6e4f83130e>", line 1, in <module>
    transactions.toPandas().drop(transactions.toPandas().columns[[4,5,6,7]], axis=1)
  File "C:\spark\python\pyspark\sql\dataframe.py", line 1703, in toPandas
    return pd.DataFrame.from_records(self.collect(), columns=self.columns)
  File "C:\spark\python\pyspark\sql\dataframe.py", line 438, in collect
    port = self._jdf.collectToPython()
  File "C:\spark\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "C:\spark\python\pyspark\sql\utils.py", line 63, in deco
    return f(*a, **kw)
  File "C:\spark\python\lib\

## 4.  Joining all features and labels together

## 5. Feature Correlation EDA to reduce redundancy

## 6. Go to KKBox website and read membership package detai to guess user churn motivation for feature engineering

## 7. Go through other one's models and learn Xgboost

## 8. Try simplest models and submit the 1st result 